In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import SimpleITK as sitk

        

2025-05-17 14:40:16.606313: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-17 14:40:16.613529: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-17 14:40:16.621559: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-17 14:40:16.624008: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-17 14:40:16.630488: I tensorflow/core/platform/cpu_feature_guar

In [4]:
model =  tf.keras.models.load_model("3D_MRI_Brain_tumor_segmentation.h5", compile=False)




I0000 00:00:1747482018.302602  464597 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1747482018.329596  464597 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1747482018.331436  464597 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1747482018.334237  464597 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [26]:
# Load the NIfTI file
img_array = np.load("dataset/ready/TCGA-DU-5852.npy")
# template_image = sitk.ReadImage("dataset/ready/UPENN-GBM-00601.nii.gz")

# # Apply Unsharp Mask to sharpen the image

# nifti_img = sitk.UnsharpMask(nifti_img)

# matcher = sitk.HistogramMatchingImageFilter()

# # Optional parameters
# matcher.SetNumberOfHistogramLevels(256)       # Number of histogram bins
# matcher.SetNumberOfMatchPoints(15)            # Number of quantile points to match
# matcher.ThresholdAtMeanIntensityOn()          # Ignore outliers by thresholding at mean intensity

# # Apply histogram matching
# matched_image = matcher.Execute(nifti_img, template_image)

# img_array = sitk.GetArrayFromImage(matched_image)  # shape: (depth, height, width)
# Normalize to [0, 1] range
img_array = img_array.astype(np.float32)
img_array = (img_array - np.min(img_array)) / (np.max(img_array) - np.min(img_array))
print(img_array.shape)

(52, 128, 128)


In [27]:
import cv2  # for resizing

predictions = []

for slice_ in img_array:
    print(slice_.shape)
    # Resize to 128x128
    # resized = cv2.resize(slice_, (128, 128), interpolation=cv2.INTER_LINEAR)
    resized = np.expand_dims(slice_, axis=-1)              # shape: (128, 128, 1)
    resized = np.repeat(resized, 2, axis=-1)                # shape: (128, 128, 2)
    resized = np.expand_dims(resized, axis=0)               # shape: (1, 128, 128, 2)

    # Predict
    pred_mask = model.predict(resized)[0, :, :, 0]

    def get_classification(pred):
        # Get the class with the highest probability for each pixel
        class_predictions = np.argmax(pred, axis=-1)
        return class_predictions

    predictions.append(get_classification(pred_mask))


(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
(128, 128)
1/1 ━━━━━

In [ ]:
import numpy as np
import cv2

scale_factor = 4
confidence_threshold = 0.10  # show overlay only if confidence >= this

for i, slice_ in enumerate(img_array):
    # Prepare input for model
    resized = np.expand_dims(slice_, axis=-1)
    resized = np.repeat(resized, 2, axis=-1)
    resized = np.expand_dims(resized, axis=0)

    pred = model.predict(resized)[0]  # (H, W, num_classes)
    confidence = np.max(pred, axis=-1)
    class_mask = np.argmax(pred, axis=-1).astype(np.uint8)

    # Resize mask and confidence to original slice size if needed
    if class_mask.shape != slice_.shape:
        class_mask = cv2.resize(class_mask, (slice_.shape[1], slice_.shape[0]), interpolation=cv2.INTER_NEAREST)
        confidence = cv2.resize(confidence, (slice_.shape[1], slice_.shape[0]), interpolation=cv2.INTER_LINEAR)

    # Original grayscale to BGR
    orig_gray = (slice_ * 255).astype(np.uint8)
    orig_bgr = cv2.cvtColor(orig_gray, cv2.COLOR_GRAY2BGR)

    # Create overlay only for confident pixels
    overlay = cv2.applyColorMap(class_mask * 85, cv2.COLORMAP_JET)

    # Create a mask for confident pixels
    confident_mask = (confidence >= confidence_threshold).astype(np.uint8)

    # Expand mask dims to match BGR shape
    confident_mask_3ch = np.repeat(confident_mask[:, :, np.newaxis], 3, axis=2)

    # Only apply overlay where confident, else original
    blended = orig_bgr.copy()
    blended[confident_mask_3ch == 1] = cv2.addWeighted(
        orig_bgr, 0.8, overlay, 0.2, 0)[confident_mask_3ch == 1]

    # Resize for display
    display_img = cv2.resize(blended, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_LINEAR)

    cv2.imshow("Confident Overlay Only", display_img)
    key = cv2.waitKey(int(1000 / fps))
    if key == 27:
        break

cv2.destroyAllWindows()




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


QObject::moveToThread: Current thread (0x36f5cc60) is not the object's thread (0x36f5bb20).
Cannot move to target thread (0x36f5cc60)

QObject::moveToThread: Current thread (0x36f5cc60) is not the object's thread (0x36f5bb20).
Cannot move to target thread (0x36f5cc60)

QObject::moveToThread: Current thread (0x36f5cc60) is not the object's thread (0x36f5bb20).
Cannot move to target thread (0x36f5cc60)

QObject::moveToThread: Current thread (0x36f5cc60) is not the object's thread (0x36f5bb20).
Cannot move to target thread (0x36f5cc60)

QObject::moveToThread: Current thread (0x36f5cc60) is not the object's thread (0x36f5bb20).
Cannot move to target thread (0x36f5cc60)

QObject::moveToThread: Current thread (0x36f5cc60) is not the object's thread (0x36f5bb20).
Cannot move to target thread (0x36f5cc60)

QObject::moveToThread: Current thread (0x36f5cc60) is not the object's thread (0x36f5bb20).
Cannot move to target thread (0x36f5cc60)

QObject::moveToThread: Current thread (0x36f5cc60) is n

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━

: 